In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['LAB']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('lactic acid bacteria', 1146),
 ('lactic acid bacterium', 13),
 ('lactic acid bacterial', 9),
 ('lactic acid produced bacteria', 9),
 ('laser assisted bioprinting', 6),
 ('low anxiety related behavior', 6),
 ('lithospermate acid b', 5),
 ('hab and low', 5),
 ('lactic bacteria', 4),
 ('linker for activity of b cells', 4),
 ('low anxiety', 4),
 ('learning ability battery', 3),
 ('l arabinitol', 3),
 ('lactalbumin', 3),
 ('ligand containing apob', 3),
 ('magnesium lithospermate b', 3)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-09 14:56:40] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'hab and low': 'ungrounded',
  'l arabinitol': 'CHEBI:CHEBI:18403',
  'lactalbumin': 'MESH:D007768',
  'lactic acid bacteria': 'MESH:D056546',
  'lactic acid bacterial': 'MESH:D056546',
  'lactic acid bacterium': 'MESH:D056546',
  'lactic acid produced bacteria': 'MESH:D056546',
  'lactic bacteria': 'MESH:D056546',
  'laser assisted bioprinting': 'MESH:D062028',
  'learning ability battery': 'ungrounded',
  'ligand containing apob': 'HGNC:603',
  'linker for activity of b cells': 'HGNC:12745',
  'lithospermate acid b': 'MESH:C061091',
  'low anxiety': 'ungrounded',
  'low anxiety related behavior': 'ungrounded',
  'magnesium lithospermate b': 'MESH:C061091'},
 {'CHEBI:CHEBI:18403': 'L-arabinitol',
  'MESH:D007768': 'Lactalbumin',
  'MESH:D056546': 'Lactobacillales',
  'MESH:D062028': 'Bioprinting',
  'HGNC:603': 'APOB',
  'HGNC:12745': 'LAT2',
  'MESH:C061091': 'lithospermate B'},
 ['HGNC:12745', 'MESH:C061091', 'MESH:D056546']]

In [6]:
grounding_map, names, pos_labels = [{'hab and low': 'ungrounded',
  'l arabinitol': 'CHEBI:CHEBI:18403',
  'lactalbumin': 'MESH:D007768',
  'lactic acid bacteria': 'MESH:D056546',
  'lactic acid bacterial': 'MESH:D056546',
  'lactic acid bacterium': 'MESH:D056546',
  'lactic acid produced bacteria': 'MESH:D056546',
  'lactic bacteria': 'MESH:D056546',
  'laser assisted bioprinting': 'MESH:D062028',
  'learning ability battery': 'ungrounded',
  'ligand containing apob': 'HGNC:603',
  'linker for activity of b cells': 'HGNC:12749',
  'lithospermate acid b': 'MESH:C061091',
  'low anxiety': 'ungrounded',
  'low anxiety related behavior': 'ungrounded',
  'magnesium lithospermate b': 'MESH:C061091'},
 {'CHEBI:CHEBI:18403': 'L-arabinitol',
  'MESH:D007768': 'Lactalbumin',
  'MESH:D056546': 'Lactobacillales',
  'MESH:D062028': 'Bioprinting',
  'HGNC:603': 'APOB',
  'HGNC:12749': 'LAT2',
  'MESH:C061091': 'lithospermate B'},
 ['HGNC:12749', 'MESH:C061091', 'MESH:D056546']]

In [7]:
excluded_longforms = []

In [8]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [9]:
additional_entities = {'HGNC:12749': ['LAT2', ['LAB']]}

In [10]:
unambiguous_agent_texts = {}

In [11]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:12749', 'HGNC:12749', 35)]

In [15]:
intersection2

[('MESH:D056546', 'HGNC:12749', 0),
 ('ungrounded', 'HGNC:12749', 0),
 ('MESH:C061091', 'HGNC:12749', 0),
 ('HGNC:12749', 'HGNC:12749', 0),
 ('HGNC:603', 'HGNC:12749', 0),
 ('MESH:D007768', 'HGNC:12749', 0),
 ('CHEBI:CHEBI:18403', 'HGNC:12749', 0),
 ('MESH:D062028', 'HGNC:12749', 0)]

In [16]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [17]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [18]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-30 02:48:01] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-30 02:48:15] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9859568013078486 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [19]:
classifier.stats

{'label_distribution': {'MESH:D056546': 875,
  'ungrounded': 11,
  'MESH:C061091': 6,
  'HGNC:12749': 13,
  'HGNC:603': 3,
  'MESH:D007768': 2,
  'CHEBI:CHEBI:18403': 2,
  'MESH:D062028': 1},
 'f1': {'mean': 0.985957, 'std': 0.003043},
 'precision': {'mean': 0.980058, 'std': 0.004385},
 'recall': {'mean': 0.992172, 'std': 0.002732},
 'ungrounded': {'f1': {'mean': 0.866667, 'std': 0.163299},
  'pr': {'mean': 0.8, 'std': 0.244949},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:C061091': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'HGNC:12749': {'f1': {'mean': 0.96, 'std': 0.08},
  'pr': {'mean': 0.933333, 'std': 0.133333},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'HGNC:603': {'f1': {'mean': 0.6, 'std': 0.489898},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'MESH:D007768': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'CHEB

In [20]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [21]:
disamb.dump(model_name, results_path)

In [22]:
print(disamb.info())

Disambiguation model for LAB

Produces the disambiguations:
	APOB	HGNC:603
	Bioprinting	MESH:D062028
	L-arabinitol	CHEBI:CHEBI:18403
	LAT2*	HGNC:12749
	Lactalbumin	MESH:D007768
	Lactobacillales*	MESH:D056546
	lithospermate B*	MESH:C061091

Class level metrics:
--------------------
Grounding      	Count	F1     
Lactobacillales*	875	0.99319
           LAT2*	 13	   0.96
     Ungrounded	 11	0.86667
lithospermate B*	  6	    0.0
           APOB	  3	    0.6
    Lactalbumin	  2	    0.4
   L-arabinitol	  2	    0.0
    Bioprinting	  1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.98596
	Precision:	0.98006
	Recall:		0.99217

* Positive labels
See Docstring for explanation



In [23]:
model_to_s3(disamb)

In [54]:
preds = disamb.disambiguate(all_texts.values())

In [55]:
texts = [text for text, pred in zip(all_texts.values(), preds) if pred[0] == 'HGNC:12749']

In [59]:
texts[1]

'An alternative scenario for BLNK recruitment involves a LAT-like molecule in B cells. This molecule, called either linker for activation of B cells (LAB) or non-T-cell activation linker (NTAL), was recently identified  [29 •• ,30 •• ] . LAB is expressed in B cells, natural killer cells, monocytes and mast cells  [30 •• ] . Similar to LAT, LAB is localized to lipid rafts and contains multiple conserved tyrosine motifs. When activated through the BCR and other immune receptors, LAB is phosphorylated, probably by Syk, and interacts with Grb2. Unlike LAT, however, LAB does not associate with PLC-γ isoforms  [29 •• ,30 •• ] . LAB is able to partially reconstitute calcium flux and Erk activation in LAT-deficient Jurkat T cells  [30 •• ] . In addition, when LAB protein levels are decreased in B cells using RNA interference, calcium flux and Erk activation are diminished  [29 •• ] . This demonstrates that LAB operates in the calcium and Ras-MAPK signaling pathways, similar to LAT. Furthermore